In [ ]:
"""
    Author: Kherin Bundhoo
    
"""

In [ ]:
# import libraries


In [ ]:
# Loading Data

In [ ]:
# Visualisation

In [ ]:
# Data Preprocessing

    ## Data cleansing:

        ### make all characters lowercase
        ### remove punctuation
    ## Tokenize
    ## Word Stemming

In [ ]:
# Feature Selection

 ## Term Frequency Inverse Document Frequency

In [ ]:
# Classifier

## Dummy
## Naive Bayes
## Logistic Regression
## Support Vector Machine
## Decision Trees
## K-means clustering

In [ ]:
# Evaluation

## Confusion Matrix
## ROC Curve and AUC
## Precision-Recall Curve
## Overfitting vs Underfitting
## k-fold Cross Validation

In [ ]:
# References & Resources